In [ ]:
from dcicutils import ff_utils
from functions.notebook_functions import *

keys = open('/Users/koray/keypairs.json', 'r').read()
my_key = json.loads(keys)['koray_data']
schema_name = get_schema_names(my_key)   

rep_url = '/search/?experiments_in_set.experiment_type=Repli-seq&type=ExperimentSetReplicate&limit=all'
repliseqs = ff_utils.search_metadata(rep_url, key = my_key)
print(len(repliseqs))
rep_uuids = [i['uuid'] for i in repliseqs]
print(len(rep_uuids))

In [ ]:
store_dict = {} 
item_uuids = []

for a_uuid in rep_uuids:
    store_dict, item_uuids = record_object(a_uuid, store_dict, item_uuids, my_key, schema_name, add_pc_wfr=True)
    print(a_uuid)
    print(len(item_uuids))
    print(len(list(set(item_uuids))))
    print()

In [ ]:
#check for existing contributing labs and run statuses

for a_type in store_dict:
    if a_type in ['quality_metric_pairsqc', 'quality_metric_fastqc', 'workflow_run_awsem', 'file_processed']:
        no_cl = 0
        jsons = store_dict[a_type]
        print(a_type, len(jsons))
        for a_json in jsons:
            if not a_json.get('contributing_labs'):
                no_cl += 1
        # make sure all runs are successful
        if a_type == 'workflow_run_awsem':
            for a_json in jsons:
                if a_json['run_status'] != 'complete':
                    print(a_json['uuid'])
        print(no_cl)
        print

In [ ]:
ma_uuid = '040d334a-2872-41ec-a282-53d062044564'

for a_type in store_dict:
    if a_type in ['quality_metric_pairsqc', 'quality_metric_fastqc', 'workflow_run_awsem', 'file_processed']:
        jsons = store_dict[a_type]
        for a_json in jsons:
            try:
                current_cont_labs = a_json['contributing_labs']
            except:
                print('no cl', a_type, a_json['uuid'])
            if ma_uuid not in current_cont_labs:
                current_cont_labs.append(ma_uuid)
                ff_utils.patch_metadata({'contributing_labs':current_cont_labs}, a_json['uuid'], key=my_key)
        print('patched',a_type, len(jsons))
                
